In [2]:
import pandas as pd
import numpy as np
import scipy
import datetime
import tqdm

**Данные о просмотрах каналов**

In [2]:
df = pd.read_csv('dataset11-30.csv')
df.start_time = pd.to_datetime(df.start_time)
df.stop_time = pd.to_datetime(df.stop_time)

**Данные о расписании ТВ-передач**

In [15]:
df_info = pd.read_csv('export_arh_11-20-final.csv').append(pd.read_csv('export_arh_21-30-final.csv'))
df_info.start_time = pd.to_datetime(df_info.start_time, format='%d.%m.%Y %H:%M:%S')
df_info['stop_time'] = df_info['start_time'] + pd.to_timedelta(df_info['duration'], unit='s')
df_info = df_info[df_info.tv_show_id != 0].copy()

**Оценка данных**

In [16]:
print('Количество пользователей:', df.user_id.nunique())
print('Количество каналов:', df.vsetv_id.nunique())
print('Количество ТВ программ:', df_info.tv_show_id.nunique())
print('Даты данных:', df.start_time.min(), df.start_time.max())
print('Даты данных info:', df_info.start_time.min(), df_info.start_time.max())

Количество пользователей: 4914
Количество каналов: 202
Количество ТВ программ: 8235
Даты данных: 2020-03-09 00:00:02 2020-07-26 23:58:42
Даты данных info: 2020-03-08 23:00:00 2020-07-27 11:55:00


**Алгоритм сбора данных о просмотрах ТВ-передач**

In [1]:
show_lines = []
for i, row in tqdm.tqdm(df.iterrows(), total = df.shape[0]):
    
    user_id = row['user_id']
    vsetv_id = row['vsetv_id']
    start_time = row['start_time']
    stop_time = row['stop_time']
    
    # Для каждой сессии пользователя найдем те передачи, которые шли на канале, который он смотрел
    shows_rememver_df = df_info[(df_info.channel_id == vsetv_id) 
                    & (df_info.start_time < stop_time) 
                    & (df_info.stop_time > start_time)][['tv_show_id', 'start_time', 'stop_time']].copy()
    
    # Запомним какие передачи просмотрел пользователь
    for j, row_j in shows_rememver_df.iterrows():
        show_lines.append([user_id, vsetv_id, start_time, stop_time, row_j['tv_show_id'], row_j['start_time'], row_j['stop_time']])
        
show_lines_df = pd.DataFrame(show_lines,columns = ['user_id', 'vsetv_id', 'start_time', 'stop_time', 'tv_show_id', 'show_start_time', 'show_stop_time'])
show_lines_df.to_csv('dataset_with_shows.csv', index = False)